In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_loc="/kaggle/input/titanic/train.csv"
train_data=pd.read_csv(train_loc)
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test_loc="/kaggle/input/titanic/test.csv"
test_data=pd.read_csv(test_loc)
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


testing and train dataset uploaded

time to define a model


In [4]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier(n_estimators=200,random_state=1)

selecting all the categorical features that can be implemented in the ML model


In [5]:
discrete_features = ["Pclass", "Sex", "SibSp", "Parch","Embarked"]
X1=pd.get_dummies(train_data[discrete_features])

In [6]:
X1.head(10)

,Pclass,SibSp,Parch,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,1,0,0,1,0,0,1
1,1,1,0,1,0,1,0,0
2,3,0,0,1,0,0,0,1
3,1,1,0,1,0,0,0,1
4,3,0,0,0,1,0,0,1
5,3,0,0,0,1,0,1,0
6,1,0,0,0,1,0,0,1
7,3,3,1,0,1,0,0,1
8,3,0,2,1,0,0,0,1
9,2,1,0,1,0,1,0,0


Float Data to be used for the classification

In [7]:
X2=train_data["Fare"]

preparing dataset for mutual information

In [8]:
X=pd.concat([X1,X2],axis=1)

In [9]:
X.head()

,Pclass,SibSp,Parch,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Fare
0,3,1,0,0,1,0,0,1,7.2500
1,1,1,0,1,0,1,0,0,71.2833
2,3,0,0,1,0,0,0,1,7.9250
3,1,1,0,1,0,0,0,1,53.1000
4,3,0,0,0,1,0,0,1,8.0500


target feature to be classified 

In [10]:
y=train_data["Survived"]

Mutual Information to check which features are most useful for the classification

uploading the package necessary

In [11]:
model.fit(X,y)

RandomForestClassifier(n_estimators=200, random_state=1)

In [12]:
X1_test=pd.get_dummies(test_data[discrete_features])

In [13]:
X2_test=test_data["Fare"]

creating the final test dataset to predict with XGBoost

In [14]:
X_test=pd.concat([X1_test,X2_test],axis=1)
X_test.head()

,Pclass,SibSp,Parch,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Fare
0,3,0,0,0,1,0,1,0,7.8292
1,3,1,0,1,0,0,0,1,7.0000
2,2,0,0,0,1,0,1,0,9.6875
3,3,0,0,0,1,0,0,1,8.6625
4,3,1,1,1,0,0,0,1,12.2875


In [15]:
X_test.isnull().sum()

Pclass        0
SibSp         0
Parch         0
Sex_female    0
Sex_male      0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
Fare          1
dtype: int64

In [16]:
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer()
imputed_X_test = pd.DataFrame(my_imputer.fit_transform(X_test))

# Imputation removed column names; put them back
imputed_X_test.columns = X_test.columns

checking for missing values after imputing

In [17]:
X_test.head()

,Pclass,SibSp,Parch,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Fare
0,3,0,0,0,1,0,1,0,7.8292
1,3,1,0,1,0,0,0,1,7.0000
2,2,0,0,0,1,0,1,0,9.6875
3,3,0,0,0,1,0,0,1,8.6625
4,3,1,1,1,0,0,0,1,12.2875


In [18]:
X_test=X_test.fillna(method='bfill', axis=0).fillna(0)

In [19]:
X_test.isnull().sum()

Pclass        0
SibSp         0
Parch         0
Sex_female    0
Sex_male      0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
Fare          0
dtype: int64

In [20]:
predictions=model.predict(X_test)

In [21]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
